In [89]:
# converting orbital elements into r and v, and vice versa

In [90]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy as sp

In [91]:
def convert_r_v(r, v): # converts position and velocity vectors, r and v, into orbital elements
    unit_i = np.array([1, 0, 0])
    unit_j = np.array([0, 1, 0])
    unit_k = np.array([0, 0, 1])
    mag_r = np.linalg.norm(r)
    mag_v = np.linalg.norm(v)
    mu = 1
    h = np.cross(r, v)
    n = np.cross(unit_k, h)
    mag_n = np.linalg.norm(n)
    mag_h = np.linalg.norm(h)
    p = mag_h**2
    e = (1 / mu) * (((mag_v**2 - (mu / mag_r)) * r) - (np.dot(r, v) * v))
    mag_e = np.linalg.norm(e)
    i = np.arccos((np.dot(h, unit_k)) / mag_h) * (180 / np.pi)
    if mag_n == 0:
        little_omega = 'none'
        big_omega = 'none'
    else:
        little_omega = np.arccos(np.dot(n, e) / (mag_n * mag_e)) * (180 / np.pi)
        big_omega = np.arccos(np.dot(n, unit_i) / mag_n) * (180 / np.pi)
    nu = np.arccos(np.dot(e, r) / (mag_e * mag_r)) * (180 / np.pi)
    return (p, mag_e, i, little_omega, big_omega, nu)

In [92]:
r = np.array([(3 * 3**(1/2)) / 4, 3 / 4, 0])
v = np.array([(-1 / (2 * 2**(1/2)), 3**(1/2) / (2 * 2**(1/2)), 1 / 2**(1/2))])
p, e, i, little_omega, big_omega, nu = convert_r_v(r, v)
print('p = ', p, 'DU\ne = ', e, '\ni = ', i, '\nlittle omega = ', little_omega, '\nbig omega = ', big_omega, '\nv = ', nu)

ValueError: shapes (3,) and (1,3) not aligned: 3 (dim 0) != 1 (dim 0)

In [79]:
def convert_orbital_perifocal(p, e, nu): # converts orbital elements into position and velocity vectors in the perifocal frame
    mu = 1
    mag_r = p / (1 + (e * np.cos(nu)))
    r = np.array([mag_r * np.cos(nu), mag_r * np.sin(nu), 0])
    v = np.array([((mu / p) ** (1 / 2)) * (-1 * np.sin(nu)), ((mu / p) ** (1 / 2)) * (e + np.cos(nu)), 0])
    return r, v

In [80]:
r, v = convert_orbital_perifocal(p, e, nu)
print ('r = ', r, '\nv = ', v)

r =  [ 2.  0.  0.] 
v =  [-0.  1.  0.]


In [83]:
def convert_orbital_geocentric(p, e, i, little_omega, big_omega, nu): # converts orbital elements into perifocal then geocentric
    r, v = convert_orbital_perifocal(p, e, nu)
    if big_omega == 'none' or little_omega == 'none':
        return r, v
    else:
        trans_matrix = np.array([[(np.cos(big_omega) * np.cos(little_omega)) - (np.sin(big_omega) * np.sin(little_omega) * np.cos(i)),
                             (-1 * np.cos(big_omega) * np.sin(little_omega)) - (np.sin(big_omega) * np.cos(little_omega) * np.cos(i)),
                             np.sin(big_omega) * np.sin(i)],
                             [(np.sin(big_omega) * np.cos(little_omega)) + (np.cos(big_omega) * np.sin(little_omega) * np.cos(i)),
                             (-1 * np.sin(big_omega) * np.sin(little_omega)) + (np.cos(big_omega) * np.cos(little_omega) * np.cos(i)),
                             -1 * np.cos(big_omega) * np.sin(i)],[np.sin(little_omega) * np.sin(i), np.cos(little_omega) * np.sin(i),
                                                                 np.cos(i)]])
        r = np.cross(trans_matrix, r)
        v = np.cross(trans_matrix, v)
    return r, v

In [87]:
r, v = convert_orbital_geocentric(p, e, i, little_omega, big_omega, nu)
print ('r = ', r, '\nv = ', v)

r =  [ 2.  0.  0.] 
v =  [-0.  1.  0.]
